In [1]:
import os
import json
import pandas as pd
import numpy as np
from pytorch_transformers import BertModel,BertTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
from tqdm import tqdm, trange


In [2]:
!nvidia-smi

Fri Apr 10 07:19:57 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:AF:00.0 Off |                    0 |
| N/A   41C    P0    59W / 300W |      0MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
from my_process_json import *

In [4]:
data_reader = MyDataReader('/home/akobtan/NLP_Project/Project/clicr_dataset/dev1.0.json')
#data_reader = MyDataReader(bs=500)

In [5]:
mydata = data_reader.send_batches()

In [6]:
dataset_size = data_reader.get_data_size()
dataset_size

3783

In [7]:
paragraphs = [e['p'] for e in mydata].copy()
paragraph_tags = [e['p_tags'] for e in mydata].copy()
queries = [e['q'] for e in mydata].copy()
query_tags = [e['q_tags'] for e in mydata].copy()

In [8]:
#creating embeddings for a batch sample from CLICR NOOOOOOO
tags_vals = ['B-ans','I-ans','O','[PAD]','[SEP]','[CLS]']
tag2idx = {t: i for i, t in enumerate(tags_vals)}

In [9]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

Using TensorFlow backend.


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [11]:
device

device(type='cuda')

In [12]:
torch.cuda.get_device_name(0) 

'Tesla V100-SXM2-16GB'

In [13]:
output_dir = "./models2/"
tokenizer = BertTokenizer.from_pretrained(output_dir, do_lower_case=True)  # Add specific options if needed

In [14]:
import copy
split_size = 200
p_tags = copy.deepcopy(paragraph_tags)
q_tags = copy.deepcopy(query_tags)
tokenized_texts = []
labels = []
for i,paragraph in enumerate(paragraphs):
    texts = []
    paragraph_tag = copy.deepcopy(p_tags[i])
    #number of splits for current paragraph
    n_splits = int(len(paragraph.split())/split_size)
    #tokenize split paragraph text appended by full query at the end with <sep> token between paragraph and query
    queries[i] = queries[i].replace("@placeholder",'B')
    texts += [s for s in ['[CLS] ' + ' '.join(paragraph.split()[(split_size*split):(split_size*(split+1))]) + ' [SEP] ' + queries[i] + ' [SEP]' for split in range(n_splits)]]
    tokenized_texts += [tokenizer.tokenize(s) for s in ['[CLS] ' + ' '.join(paragraph.split()[(split_size*split):(split_size*(split+1))]) + ' [SEP] ' + queries[i] + ' [SEP]' for split in range(n_splits)]]
    #tokenize remainder of splits
    if int(len(paragraph.split()) % split_size) > 0:
        texts += ['[CLS] ' + ' '.join(paragraph.split()[(n_splits*split_size):]) + ' [SEP] ' + queries[i] + ' [SEP]']
        tokenized_texts += [tokenizer.tokenize('[CLS] ' + ' '.join(paragraph.split()[(n_splits*split_size):]) + ' [SEP] ' + queries[i] + ' [SEP]')]
    
    #create labels to be used for tagging from the text
    for split_index, sent in enumerate(texts):
        l = []
        for word_index,word in enumerate(sent.split()):
            if '[SEP]' in word:
                l += ['[SEP]']
                break
            if '[CLS]' in word:
                l += ['[CLS]']
                continue
            if not paragraph_tag:
                print(sent,'\n')
                print(word)
                print(split_index)
            lab = paragraph_tag.pop(0)
            word_list = tokenizer.tokenize(word)
            for w in word_list:
                l += [lab]
        query_tags = copy.deepcopy(q_tags[i])
        for word in queries[i].split():
            lab = query_tags.pop(0)
            word_list = tokenizer.tokenize(word)
            for w in word_list:
                l += [lab]
        l += ['[SEP]']
        labels += [l]
            

In [15]:
len(labels[0])

279

In [16]:
len(tokenized_texts[0])

279

In [17]:
max([len(t) for t in tokenized_texts])

415

In [18]:
max([len(t) for t in labels])

415

In [19]:
print(tokenized_texts[0])

['[CLS]', 'caf', '##fe', '##y', 'disease', 'in', 'neon', '##atal', 'period', ':', 'the', 'importance', 'of', 'the', 'family', '!', 'summary', 'a', 'male', 'newborn', 'was', 'apparently', 'well', 'until', 'his', 'second', 'day', 'of', 'life', ',', 'when', 'increased', 'ir', '##rita', '##bility', 'and', 'a', 'swelling', 'in', 'his', 'right', 'leg', 'were', 'noted', '.', 'he', 'was', 'room', '##ing', '-', 'in', 'with', 'his', 'mother', 'since', 'birth', '.', 'on', 'examination', ',', 'a', 'mass', 'on', 'the', 'anterior', 'surface', 'of', 'the', 'right', 'leg', 'was', 'noticed', '.', 'the', 'mass', 'was', 'firm', ',', 'elongated', ',', 'ill', '-', 'defined', ',', 'un', '##movable', 'and', 'painful', 'at', 'pal', '##pati', '##on', '.', 'no', 'overly', '##ing', 'skin', 'changes', 'were', 'seen', '.', 'the', 'newborn', 'had', 'a', 'family', 'history', 'of', 'neon', '##atal', 'bone', 'swelling', 'with', 'resolution', 'before', 'the', 'age', 'of', '2', '.', 'subsequent', 'images', 'showed', 'hy

In [20]:
print(labels[0])

['[CLS]', 'B-ans', 'B-ans', 'B-ans', 'I-ans', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ans', 'B-ans', 'B-ans', 'I-ans', 'O'

In [21]:
DATASET_SIZE = 3783
MAX_LEN = 424
SEED = 520
bs = 16

In [22]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
input_ids[0]

array([  101, 24689,  7959,  2100,  4295,  1999, 16231, 27815,  2558,
        1024,  1996,  5197,  1997,  1996,  2155,   999, 12654,  1037,
        3287, 20662,  2001,  4593,  2092,  2127,  2010,  2117,  2154,
        1997,  2166,  1010,  2043,  3445, 20868, 17728,  8553,  1998,
        1037, 18348,  1999,  2010,  2157,  4190,  2020,  3264,  1012,
        2002,  2001,  2282,  2075,  1011,  1999,  2007,  2010,  2388,
        2144,  4182,  1012,  2006,  7749,  1010,  1037,  3742,  2006,
        1996, 15099,  3302,  1997,  1996,  2157,  4190,  2001,  4384,
        1012,  1996,  3742,  2001,  3813,  1010, 17876,  1010,  5665,
        1011,  4225,  1010,  4895, 25661,  1998,  9145,  2012, 14412,
       24952,  2239,  1012,  2053, 15241,  2075,  3096,  3431,  2020,
        2464,  1012,  1996, 20662,  2018,  1037,  2155,  2381,  1997,
       16231, 27815,  5923, 18348,  2007,  5813,  2077,  1996,  2287,
        1997,  1016,  1012,  4745,  4871,  3662, 23760, 14122, 12650,
        1999,  1996,

In [23]:
len(input_ids)

33701

In [24]:
len(input_ids[0])

424

In [25]:
token_type_ids = []
for ipid in input_ids:
    type_id = 0
    token_type_id = []
    for myid in ipid:
        token_type_id.append(type_id)
        if myid == 102: 
            if type_id%2==0:
                type_id+= 1
            else:
                type_id=0
    token_type_ids.append(token_type_id)
    

In [26]:
print(token_type_ids[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [27]:
len(token_type_ids[0])

424

In [28]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["[PAD]"], padding="post",
                     dtype="long", truncating="post")


In [29]:
tags[0]

array([5, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [30]:
len(tags)

33701

In [31]:
len(tags[0])

424

In [32]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]
print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

In [33]:
val_inputs = torch.tensor(input_ids)
val_tags = torch.tensor(tags)
val_masks = torch.tensor(attention_masks)

In [34]:
torch.cuda.empty_cache() 

In [35]:
valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [36]:
!nvidia-smi

Fri Apr 10 07:24:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:AF:00.0 Off |                    0 |
| N/A   36C    P0    43W / 300W |     11MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [37]:
output_dir = "./models2/"
# Step 2: Re-load the saved model and vocabulary

# Example for a Bert model
model = BertForTokenClassification.from_pretrained(output_dir,num_labels=len(tag2idx))

In [38]:
model.cuda();

In [39]:
!nvidia-smi

Fri Apr 10 07:24:54 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:AF:00.0 Off |                    0 |
| N/A   37C    P0    58W / 300W |   1472MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [40]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=1e-4)

In [41]:
from seqeval.metrics import f1_score
from seqeval.metrics import classification_report

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [42]:
model.eval()
predictions = []
true_labels = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                              attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None,
                       attention_mask=b_input_mask)
        
    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    label_ids = b_labels.to('cpu').numpy()
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
valid_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]
print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(valid_tags, pred_tags)))
rep = classification_report(valid_tags, pred_tags)
print(rep)

Validation loss: 0.1230558043477522
Validation Accuracy: 0.6465969790724543
Validation F1-Score: 0.6606031478307656
           precision    recall  f1-score   support

    [PAD]       0.00      0.00      0.00     33701
    [CLS]       0.98      1.00      0.99     33701
      ans       0.21      0.15      0.17     55631
    [SEP]       1.00      1.00      1.00     67402

micro avg       0.78      0.57      0.66    190435
macro avg       0.59      0.57      0.58    190435



In [43]:
test_reader = MyDataReader('/home/akobtan/NLP_Project/Project/clicr_dataset/test1.0.json')
#data_reader = MyDataReader(bs=500)

In [44]:
testdata = test_reader.send_batches()

In [45]:
testset_size = test_reader.get_data_size()
testset_size

4147

In [46]:
testparagraphs = [e['p'] for e in testdata].copy()
testparagraph_tags = [e['p_tags'] for e in testdata].copy()
testqueries = [e['q'] for e in testdata].copy()
testquery_tags = [e['q_tags'] for e in testdata].copy()

In [47]:
#creating embeddings for a batch sample from CLICR NOOOOOOO
testtags_vals = ['B-ans','I-ans','O','[PAD]','[SEP]','[CLS]']
testtag2idx = {t: i for i, t in enumerate(testtags_vals)}

In [56]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

In [48]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [49]:
device

device(type='cuda')

In [50]:
torch.cuda.get_device_name(0) 

'Tesla V100-SXM2-16GB'

In [51]:
import copy
split_size = 200
p_tags = copy.deepcopy(testparagraph_tags)
q_tags = copy.deepcopy(testquery_tags)
testtokenized_texts = []
testlabels = []
for i,paragraph in enumerate(testparagraphs):
    texts = []
    paragraph_tag = copy.deepcopy(p_tags[i])
    #number of splits for current paragraph
    n_splits = int(len(paragraph.split())/split_size)
    #tokenize split paragraph text appended by full query at the end with <sep> token between paragraph and query
    testqueries[i] = testqueries[i].replace("@placeholder",'B')
    texts += [s for s in ['[CLS] ' + ' '.join(paragraph.split()[(split_size*split):(split_size*(split+1))]) + ' [SEP] ' + testqueries[i] + ' [SEP]' for split in range(n_splits)]]
    testtokenized_texts += [tokenizer.tokenize(s) for s in ['[CLS] ' + ' '.join(paragraph.split()[(split_size*split):(split_size*(split+1))]) + ' [SEP] ' + testqueries[i] + ' [SEP]' for split in range(n_splits)]]
    #tokenize remainder of splits
    if int(len(paragraph.split()) % split_size) > 0:
        texts += ['[CLS] ' + ' '.join(paragraph.split()[(n_splits*split_size):]) + ' [SEP] ' + testqueries[i] + ' [SEP]']
        testtokenized_texts += [tokenizer.tokenize('[CLS] ' + ' '.join(paragraph.split()[(n_splits*split_size):]) + ' [SEP] ' + testqueries[i] + ' [SEP]')]
    
    #create labels to be used for tagging from the text
    for split_index, sent in enumerate(texts):
        l = []
        for word_index,word in enumerate(sent.split()):
            if '[SEP]' in word:
                l += ['[SEP]']
                break
            if '[CLS]' in word:
                l += ['[CLS]']
                continue
            if not paragraph_tag:
                print(sent,'\n')
                print(word)
                print(split_index)
            lab = paragraph_tag.pop(0)
            word_list = tokenizer.tokenize(word)
            for w in word_list:
                l += [lab]
        query_tags = copy.deepcopy(q_tags[i])
        for word in testqueries[i].split():
            lab = query_tags.pop(0)
            word_list = tokenizer.tokenize(word)
            for w in word_list:
                l += [lab]
        l += ['[SEP]']
        testlabels += [l]
            

In [52]:
len(testlabels[0])

287

In [53]:
len(testtokenized_texts[0])

287

In [54]:
max([len(t) for t in testtokenized_texts])

400

In [55]:
max([len(t) for t in testlabels])

400

In [56]:
print(testtokenized_texts[0])

['[CLS]', 'more', 'than', 'meets', 'the', 'eye', ':', 'the', '‘', 'pink', 'salmon', 'patch', '’', 'summary', 'o', '##cular', 'ad', '##ne', '##xa', '##l', 'l', '##ym', '##ph', '##oma', '##s', 'account', 'for', '1', '–', '2', '%', 'of', 'all', 'non', '-', 'ho', '##d', '##g', '##kin', "'", 's', 'l', '##ym', '##ph', '##oma', '##s', '.', 'con', '##jun', '##ct', '##iva', 'is', 'the', 'primary', 'site', 'of', 'involvement', 'in', 'one', '-', 'third', 'of', 'cases', '.', 'we', 'present', 'a', 'case', 'of', 'a', '47', '-', 'year', '-', 'old', 'hispanic', 'woman', 'who', 'presented', 'with', 'left', 'eye', 'it', '##ching', 'and', 'irritation', 'associated', 'with', 'a', 'pain', '##less', 'pink', 'mass', '.', 'physical', 'examination', 'revealed', 'the', 'presence', 'of', 'a', '‘', 'pink', 'salmon', '-', 'patch', '’', 'involving', 'her', 'left', 'medial', 'con', '##jun', '##ct', '##iva', '.', 'orbital', 'ct', 'showed', 'a', 'sub', '##cent', '##imeter', 'left', 'pre', '##se', '##pta', '##l', 'soft

In [57]:
print(labels[0])

['[CLS]', 'B-ans', 'B-ans', 'B-ans', 'I-ans', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ans', 'B-ans', 'B-ans', 'I-ans', 'O'

In [58]:
DATASET_SIZE = 4147
MAX_LEN = 424
SEED = 520
bs = 16

In [59]:
testinput_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in testtokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
testinput_ids[0]

array([  101,  2062,  2084,  6010,  1996,  3239,  1024,  1996,  1520,
        5061, 11840,  8983,  1521, 12654,  1051, 15431,  4748,  2638,
       18684,  2140,  1048, 24335,  8458,  9626,  2015,  4070,  2005,
        1015,  1516,  1016,  1003,  1997,  2035,  2512,  1011,  7570,
        2094,  2290,  4939,  1005,  1055,  1048, 24335,  8458,  9626,
        2015,  1012,  9530, 19792,  6593, 11444,  2003,  1996,  3078,
        2609,  1997,  6624,  1999,  2028,  1011,  2353,  1997,  3572,
        1012,  2057,  2556,  1037,  2553,  1997,  1037,  4700,  1011,
        2095,  1011,  2214,  6696,  2450,  2040,  3591,  2007,  2187,
        3239,  2009,  8450,  1998, 17373,  3378,  2007,  1037,  3255,
        3238,  5061,  3742,  1012,  3558,  7749,  3936,  1996,  3739,
        1997,  1037,  1520,  5061, 11840,  1011,  8983,  1521,  5994,
        2014,  2187, 23828,  9530, 19792,  6593, 11444,  1012, 13943,
       14931,  3662,  1037,  4942, 13013, 19198,  2187,  3653,  3366,
       22799,  2140,

In [60]:
len(testinput_ids)

34737

In [61]:
len(testinput_ids[0])

424

In [62]:
testtoken_type_ids = []
for ipid in input_ids:
    type_id = 0
    token_type_id = []
    for myid in ipid:
        token_type_id.append(type_id)
        if myid == 102: 
            if type_id%2==0:
                type_id+= 1
            else:
                type_id=0
    testtoken_type_ids.append(token_type_id)
    

In [63]:
print(testtoken_type_ids[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [64]:
len(testtoken_type_ids[0])

424

In [65]:
testtags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in testlabels],
                     maxlen=MAX_LEN, value=tag2idx["[PAD]"], padding="post",
                     dtype="long", truncating="post")


In [ ]:
testtags[0]

array([5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [ ]:
len(testtags)

34737

In [ ]:
len(testtags[0])

424

In [ ]:
testattention_masks = [[float(i>0) for i in ii] for ii in testinput_ids]
print(testattention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

In [ ]:
test_inputs = torch.tensor(testinput_ids)
test_tags = torch.tensor(testtags)
test_masks = torch.tensor(testattention_masks)

In [ ]:
torch.cuda.empty_cache() 

In [ ]:
test_data = TensorDataset(test_inputs, test_masks, test_tags)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=bs)

In [ ]:
!nvidia-smi

Thu Apr  9 23:40:49 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:AF:00.0 Off |                    0 |
| N/A   39C    P0    58W / 300W |   1520MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=1e-4)

In [ ]:
model.eval()
predictions = []
true_labels = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
for batch in test_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                              attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None,
                       attention_mask=b_input_mask)
        
    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    label_ids = b_labels.to('cpu').numpy()
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
valid_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]
print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(valid_tags, pred_tags)))
rep = classification_report(valid_tags, pred_tags)
print(rep)

Validation loss: 0.09582172647884434
Validation Accuracy: 0.6478543925257987
